In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
training_epoch= 15
batch_size =100


In [3]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name): #모델별 w 가 충돌하지말라고 scope(범위)를 지정해줌
            self.training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, shape = [None, 784])
            X_img = tf.reshape(self.X, [-1,28,28,1])
            self.Y = tf.placeholder(tf.float32, shape = [None, 10])
            
            conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size=[3,3], padding='SAME',
                                    activation = tf.nn.relu) 
            #tf 에 있는 컨볼루젼 모듈, relu까지 통과한 값을 리턴함
            
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size=[2,2],
                                       padding = 'SAME', strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7,
                                        training=self.training) #true면 먹이고(0.7), false면 안맥임(1)
            
            
            conv2 = tf.layers.conv2d(inputs =dropout1 , filters = 64, kernel_size=[3,3], padding='SAME',
                                    activation = tf.nn.relu)  
            pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2],
                                       padding = 'SAME', strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7,
                                        training=self.training) 
            
            
            conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size=[3,3], padding='SAME',
                                    activation = tf.nn.relu)   
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size=[2,2],
                                       padding = 'SAME', strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.7,
                                        training=self.training) #트루면 먹이고 false면 안맥임
            
            flat = tf.reshape(dropout3, [-1,4*4*128])
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,rate=0.5,
                                        training=self.training)
            
            self.logits = tf.layers.dense(inputs = dropout4, units=10)
        
        self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels=self.Y))
        self.optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits,1), tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        def predict(self, x_test, training=False):
            return self.sess.run(self.logits, feed_dict={self.X:x_test,
                                                     self.training:training})
    
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X:x_test, self.Y:y_test, 
                                                       self.training:training})
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X:x_data,
                                                                    self.Y:y_data,
                                                                    self.training:training})
    

In [ ]:
sess = tf.Session()
models =[]
num_models = 7

for m in range(num_models):
    models.append(Model(sess, 'model'+str(m)))

sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples /batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx]+= c/total_batch
    print("Epoch : ",epoch, "Cost : ", avg_cost_list)

test_size = len(mnist.test.labels)
predictions = np.zeros([test_size,10])

for m_idx, m in enumerate(models):
    p = m.predict(mnist.test.images)
    predictions +=p

ensemble_correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(mnist.test.labels,1))

ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print(sess.run(ensemble_accuracy))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch :  0 Cost :  [0.80114747 0.76071279 0.81636639 0.76617134 0.76927571 0.78376259
 0.78393275]
Epoch :  1 Cost :  [0.30893404 0.28806822 0.30284273 0.28473615 0.29262927 0.29389012
 0.29402833]
Epoch :  2 Cost :  [0.2381475  0.22924576 0.2349116  0.22734711 0.23308485 0.23276546
 0.23387244]
Epoch :  3 Cost :  [0.20511253 0.19988612 0.2041476  0.1944822  0.20438358 0.20366167
 0.20686373]
Epoch :  4 Cost :  [0.18562719 0.18031059 0.1884738  0.18229264 0.18446193 0.18488983
 0.18637829]
Epoch :  5 Cost :  [0.17000484 0.17095886 0.1724265  0.16910357 0.17653662 0.17353596
 0.17323688]
Epoch :  6 Cost :  [0.16441778 0.16230413 0.160712   0.15564505 0.16245767 0.16463997
 0.16580645]
Epoch :  7 Cost :  [0.15737796 0.15485808 0.16029376 0.154879   0.15700971 0.15860447
 0.15987236]
Epoch :  8 Cost 